In [20]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
def build_embeddings_for_resumes(df, model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"):
    """
    Принимает DataFrame df (с колонкой 'text'), возвращает:
    1) Тот же df, но с дополнительным столбцом 'embedding'.
    2) Объект модели (SentenceTransformer), чтобы потом использовать для поиска.
    """
    # Загружаем модель
    model = SentenceTransformer(model_name)

    # Строим эмбеддинги для каждого резюме
    texts = df["text"].tolist()
    embeddings = model.encode(texts, show_progress_bar=True)

    # Записываем эмбеддинги в DataFrame
    df["embedding"] = list(embeddings)

    return df, model

In [22]:
def search_resumes_by_query(df, model, query, top_k=5, similarity_threshold=None):
    """
    Ищет наиболее релевантные резюме по запросу 'query'.
    - df должен содержать столбец 'embedding'
    - model – это SentenceTransformer (BERT)
    - top_k – сколько верхних (самых похожих) результатов вывести
    - similarity_threshold – для вывода только тех у которых
      похожесть не ниже заданного порога

    Возвращает DataFrame из найденных совпадений,
    добавляя столбец 'similarity' (косинусная близость).
    """

    # Получаем эмбеддинг для запроса
    query_embedding = model.encode([query])  # список из одного запроса
    query_embedding = query_embedding[0].reshape(1, -1)  # форма (1, dim)

    # Собираем все эмбеддинги из DataFrame
    resume_embeddings = list(df["embedding"].values)

    # Превратим resume_embeddings в массив numpy (shape: [N, dim])
    import numpy as np
    resume_embeddings = np.vstack(resume_embeddings)  # (N, dim)

    # Вычисляем косинусную близость
    similarities = cosine_similarity(query_embedding, resume_embeddings)

    # similarities имеет форму (1, N), где N – число резюме
    similarities = similarities[0]  # превращаем в shape (N, )

    df_result = df.copy()
    df_result["similarity"] = similarities

    # Сортируем по убыванию
    df_result = df_result.sort_values(by="similarity", ascending=False)

    # Если задан порог, то фильтруем
    if similarity_threshold is not None:
        df_result = df_result[df_result["similarity"] >= similarity_threshold]

    # Если задан top_k, то берем первые top_k строк
    if top_k is not None:
        df_result = df_result.head(top_k)

    return df_result

In [ ]:
df = pd.read_excel("files_text.xls")

# Создаём эмбеддинги
df, model = build_embeddings_for_resumes(df)

# Формируем поисковый запрос
query = "Опыт в data science, Python, машинном обучении"

# Ищем резюме
df_found = search_resumes_by_query(df, model, query, top_k=3, similarity_threshold=0.5)

print("Найденные резюме:")
print(df_found[["filename", "similarity"]])